### Import library

In [1]:
import pandas as pd
import cv2 
import numpy as np
import threading

from keras.models import  Model
from keras.layers import (
    Conv2D, MaxPooling2D, Activation, Dense, concatenate, AveragePooling2D,Input,GlobalAveragePooling2D
)
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.metrics import categorical_accuracy
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

### Read data

In [2]:
# đọc data
data=pd.read_csv("data/label_data.csv")
# lấy danh sách các path image
paths = list(data['pixel'])
labels = list(data['label'])
# hàm đọc ảnh để dùng cho threading
# result_list để chứa các pixel được đọc từ imge_path
def read_and_append(image_path,label,result_list,label_list):
    image = cv2.imread(image_path)
    if image is not None:
        result_list.append(image)
        label_list.append(label)
threads = [] # list chứa các thread
images = [] # list chứa các pixel của image
label_list = []
for i in range(len(paths)):
    path = paths[i]
    label = labels[i]
    thread = threading.Thread(target=read_and_append,args=(path,label,images,label_list)) # tạo một thread để thực hiện nhiệm vụ của hàm read_and_append
    threads.append(thread)
    thread.start() # bắt đầu thực hiện thread
# kiểm tra các thread đã thực hiện xong chưa
for thread in threads:
    thread.join()
df = pd.DataFrame({'images':images,'label':label_list})
del images, threads, label_list, paths, labels, data

### Resize data

In [3]:
max_width = 0
max_height = 0
for i in df['images']:
    height = i.shape[0]
    width = i.shape[1]
    if height > max_height:
        max_height = height
    if width > max_width:
        max_width = width
new_images = []
threads = []
def resized_image(pixel,max_width,max_height,list_image_resized):
    rs = np.zeros((max_height,max_width,3),dtype=np.uint8)
    height = pixel.shape[0]
    width = pixel.shape[1]
    rs[:height,:width] = pixel
    list_image_resized.append(rs)
for i in df['images']:
    thread = threading.Thread(target=resized_image,args=(i,max_width,max_height,new_images))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
df['images']=new_images
del new_images, threads

## Training 

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['images'],df['label'],train_size=0.8,random_state= 42)
del df

In [6]:
print("y_train_0:",(y_train==0).sum(),"y_train_1:",(y_train==1).sum())
print("y_test_0:",(y_test==0).sum(),"y_test_1:",(y_test==1).sum())

y_train_0: 1154 y_train_1: 1178
y_test_0: 298 y_test_1: 285


### DenseNet 

In [7]:
## create conv_block
def conv_block(x, num_chanels):
    # BatchNormalization
    x = BatchNormalization()(x)
    # add activation layer (Relu)
    x = Activation('relu')(x)
    x = Conv2D(num_chanels,(3,3), padding='same')(x)
    return x
## create Dense_block
def dense_block(x,num_layers,num_chanels):
    Y = x
    for _ in range(num_layers):
        X = conv_block(x,num_chanels)
        Y = concatenate([Y,X],axis=-1)
    return Y
## create transition block
def transition_block(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(x.shape[-1]//2,(1,1), padding = 'same')(x)
    x = AveragePooling2D()(x)
    return x


In [9]:
def DenseNet(input_shape, num_classes, num_block, num_layers_in_block, growth_rate):
    inputs = Input(shape=input_shape)
    # init first block
    x = Conv2D(2 * growth_rate,(7,7),strides=(2,2),padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3,3),strides=(2,2),padding='same')(x)
    
    for _ in range(num_block):
        x = dense_block(x , num_layers= num_layers_in_block, num_chanels = growth_rate)
        x = transition_block(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs = inputs, outputs = outputs)
    return model
# thiết lập thông số mô hình
input_shape = (500, 500, 3)
num_classes = 2
num_layer_in_block = 4
growth_rate = 32

# Tạo mô hình DenseNet
model = DenseNet(input_shape,num_classes, 4, num_layer_in_block, growth_rate)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=categorical_crossentropy,
              metrics=[categorical_accuracy])
model.summary()
    

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 500, 500, 3)]        0         []                            
                                                                                                  
 conv2d_21 (Conv2D)          (None, 250, 250, 64)         9472      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_21 (Ba  (None, 250, 250, 64)         256       ['conv2d_21[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_21 (Activation)  (None, 250, 250, 64)         0         ['batch_normalization_21[0

### Train

In [10]:
X_tr=X_train.to_list()
del X_train
X_train_new=np.array(X_tr)
del X_tr
X_t=X_test.to_list()
del X_test
X_test_new=np.array(X_t)
del X_t
y_train = y_train.to_list()
y_train_new =np.array(y_train)
y_train_new = to_categorical(y_train_new, num_classes=2)
del y_train
y_test = y_test.to_list()
y_test_new = np.array(y_test)
y_test_new = to_categorical(y_test_new, num_classes=2)
del y_test

In [11]:
model.fit(X_train_new, y_train_new,epochs=5, batch_size=32, validation_data=(X_train_new, y_train_new))

Epoch 1/5
73/73 [==============================] - 1176s 16s/step - loss: 0.7260 - categorical_accuracy: 0.5476 - val_loss: 5.3623 - val_categorical_accuracy: 0.4949
Epoch 2/5
73/73 [==============================] - 3701s 51s/step - loss: 0.6934 - categorical_accuracy: 0.5695 - val_loss: 0.7145 - val_categorical_accuracy: 0.5673
Epoch 3/5
73/73 [==============================] - 1584s 22s/step - loss: 0.6765 - categorical_accuracy: 0.5888 - val_loss: 0.7277 - val_categorical_accuracy: 0.5455
Epoch 4/5
32/73 [============>.................] - ETA: 11:52 - loss: 0.6692 - categorical_accuracy: 0.6104

In [8]:
model.save('my_model')